In [96]:
import json
import numpy as np

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]

test_file = ["sample1_period1.txt", "sample2_period2.txt","sample3_period3.txt","sample4_period1.txt",\
            "sample5_period1.txt","sample6_period2.txt","sample7_period3.txt","sample8_period1.txt", \
            "sample9_period2.txt","sample10_period3.txt",]

for i, fl in enumerate(files):
    print "files[" + str(i) + "] => " + fl

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [2]:
# transform the 2015 Feb. 1, 8:00 am and 8:00 pm into timestamp form
import datetime, time
import pytz

start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))
# used as the zero point of the time
base_time = time.mktime(time.strptime("2015-01-01 00:00:00",'%Y-%m-%d %H:%M:%S'))

In [3]:
def floor_date(date):
    return date // 3600

In [30]:
# the extracted data contains
# 0. the citation date 1. the favourite_count 2. the followers number
# 3. the length of the title 4. the number of the twitters
def load_file(file, path = "./tweet_data"):
    '''
    load the time, follows, and retweets
    '''
    # 0. before Feb. 1, 8:00 a.m. 1. between 2. after 8:00 p.m.
    result = []
    for line in open(path + file, 'r') :
        tmp = []
        a = json.loads(line)
        citation_date = a['citation_date']
        tmp.append(int((citation_date - base_time) // 3600))
        tmp.append(a['author']['followers'])
        tmp.append(a['tweet']['user']['favourites_count'])
        tmp.append(len(a['title']))
        tmp.append(1)
        result.append(tmp)
    return result

In [11]:
def group_data(dataset):
    '''
    parameter dataset is one of the three time slot defined above, namely, data[0], data[1], data[2]
    count all the features in 1-hour slot
    
    return the train feature (train_set[0:4]) and the result (train_set[5])
    '''
    # get the maximum and minimum time
    max_time = dataset[0][0]
    min_time = dataset[0][0]
    for p in dataset:
        time = p[0]
        if max_time < time:
            max_time = time
        if min_time > time:
            min_time = time
    
    train_set = []
    
    for i in range(min_time, max_time + 1):
        train_set.append([0] * 5)
    
    for p in dataset:
        index = p[0] - min_time
        train_set[index][0] = p[0]
        for i in range(1, 5):
            train_set[index][i] += p[i]
    
    result = []
    for i in range(0, max_time - min_time):
        result.append(train_set[i + 1][4])
    
    return train_set[:-1], result

In [12]:
def get_error(predict, target):
    '''
        used to get the average error between predict and target
    '''
    length = len(target)
    return sum([abs(predict[i] - target[i]) for i in range(length)]) / float(length)

In [13]:
total_file = []

for ff in files:
    total_file.append(load_file("tweets_#gopatriots.txt"))

In [19]:
train, result = group_data(total_file)

In [40]:
train_tt = []
result_tt = []
tmp = [0] * 5

for i, v in enumerate(train):
    if i % 5 == 0 and i < len(train) - 1:
        result_tt.append(result[i + 1])
        train_tt.append(tmp)
        tmp = v
    else:
        for k in range(5):
            tmp[k] += v[k]

In [61]:
from sklearn import linear_model

lr = linear_model.LinearRegression()
model = lr.fit(train, result)

In [113]:
import math

for t in test_file:
    p = load_file(t, "./test_data/")
    test_tt, result_tt = group_data(p)
    
    test_feature = [0] * 5
    test_result = 0
    
    # find the largest time
    max_time = 0
    for p in test_tt:
        max_time = max(max_time, p[0])
    
    for i, p in enumerate(test_tt):
        test_result += result_tt[i]
        
        for k in range(5):
            test_feature[k] += p[k]
                
   #print test_feature
    predict = abs(model.predict([test_feature]))
    
    print('predict value:%s'%predict[0])
    print ('actual value:%s'%test_result)
    print ""

predict value:954.6026690346104
actual value:595

predict value:6268.288017288751
actual value:204746

predict value:2474.261899333979
actual value:3188

predict value:410.6939920559822
actual value:1228

predict value:1367.9340393605019
actual value:1718

predict value:111947.98601000413
actual value:204599

predict value:65.41886829541272
actual value:403

predict value:203.0319643273405
actual value:180

predict value:6626.48115244231
actual value:9582

predict value:239.79483169725566
actual value:303

